In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:

exames_df = pd.read_csv('HC_EXAMES_1.csv', sep='|')
pacientes_df = pd.read_csv('HC_PACIENTES_1.csv',sep='|')

print("Primeiras linhas de HC_EXAMES_1.csv:")
display(exames_df.head())

print("\nPrimeiras linhas de HC_PACIENTES_1.csv:")
display(pacientes_df.head())


Primeiras linhas de HC_EXAMES_1.csv:


,ID_PACIENTE,ID_aTENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
0,004056afeb7b5441855846349123f686,0ae989c74676b6b5b8bf1a5f57be45f7,2020-05-17,HOSP,CULTURA AERÓBIA,CULTURA AERÓBIA,Negativa,NaN,Negativa
1,004056afeb7b5441855846349123f686,0ae989c74676b6b5b8bf1a5f57be45f7,2020-05-17,HOSP,URÉIA,URÉIA,10,mg/dL,10 a 50 mg/dL
2,004056afeb7b5441855846349123f686,0ae989c74676b6b5b8bf1a5f57be45f7,2020-05-17,HOSP,URINA TIPO 1,pH :,7.0,NaN,5.0 a 6.0
3,004056afeb7b5441855846349123f686,0ae989c74676b6b5b8bf1a5f57be45f7,2020-05-17,HOSP,URINA TIPO 1,Urobilinogenio :,0.2,mg/dL,0.2 a 1.0 mg/dL
4,004056afeb7b5441855846349123f686,0ae989c74676b6b5b8bf1a5f57be45f7,2020-05-17,HOSP,URINA TIPO 1,Sangue :,Tracos,NaN,Ausente



Primeiras linhas de HC_PACIENTES_1.csv:


,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO,CD_PAIS,CD_UF,CD_MUNICIPIO,CD_CEPREDUZIDO
0,9698838a8fa8a01ffd5ed5c71e8e17a3,F,1962,BR,SP,SAO PAULO,CCCC
1,d9fec23b3820f93a961841d569db8cb5,F,1974,BR,SP,MMMM,CCCC
2,ee507ba3a9959fdf31bca52852fd5715,F,1962,BR,SP,MMMM,CCCC
3,51590e8c53f4e8e332c05d7e6cee35c7,F,1960,BR,SP,SAO PAULO,CCCC
4,13699f0f7714fdaba277c5e360c6869c,M,1967,BR,SP,MMMM,CCCC


In [ ]:
exames_df_sorted = exames_df.sort_values(by=['ID_aTENDIMENTO', 'DE_EXAME', 'DT_COLETA'])
exames_df_cleaned = exames_df_sorted.drop_duplicates(subset=['ID_aTENDIMENTO', 'DE_EXAME'], keep='last')

print("Número de linhas após remover duplicatas:", len(exames_df_cleaned))
display(exames_df_cleaned.head())

Número de linhas após remover duplicatas: 135500


,ID_PACIENTE,ID_aTENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
1281318,f88a18f43c87d534877bda69ff89b068,005dc13e19b2523e0db4bf15156d6412,2020-05-10,HOSP,ALANINA AMINOTRANSFERASE,ALANINA AMINOTRANSFERASE,84,U/L,Homens : inferior a 41 U/L
1281317,f88a18f43c87d534877bda69ff89b068,005dc13e19b2523e0db4bf15156d6412,2020-05-10,HOSP,ALBUMINA,ALBUMINA,2.3,g/dL,Acima de 19 anos 3.4 a 4.8 g/dL
1281901,f88a18f43c87d534877bda69ff89b068,005dc13e19b2523e0db4bf15156d6412,2020-05-20,HOSP,ANTIBIOGRAMA,ANTIBIOGRAMA,R,NaN,
1281316,f88a18f43c87d534877bda69ff89b068,005dc13e19b2523e0db4bf15156d6412,2020-05-10,HOSP,ASPARTATO AMINO TRANSFERASE,ASPARTATO AMINO TRANSFERASE,154,U/L,Homens : inferior a 37 U/L
1281886,f88a18f43c87d534877bda69ff89b068,005dc13e19b2523e0db4bf15156d6412,2020-05-20,HOSP,BILIRRUBINA DIRETA,BILIRRUBINA DIRETA,0.51,mg/dL,inferior a 0.30 mg/dL


In [ ]:
covid_performed_df = exames_df_cleaned[exames_df_cleaned['DE_EXAME'] == 'COVID-19 - PESQUISA DE ANTICORPOS IgG']

print(f"Número de atendimentos com o exame 'COVID-19 - PESQUISA DE ANTICORPOS IgG' realizado: {len(covid_performed_df)}")

print("\nPrimeiras linhas do dataframe filtrado para o exame de COVID-19:")
display(covid_performed_df.head())

Número de atendimentos com o exame 'COVID-19 - PESQUISA DE ANTICORPOS IgG' realizado: 554

Primeiras linhas do dataframe filtrado para o exame de COVID-19:


,ID_PACIENTE,ID_aTENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
1173251,d468b9d76f04d517bc4c4140744685ba,016586ae084a86326b913fe74146b102,2020-05-30,HOSP,COVID-19 - PESQUISA DE ANTICORPOS IgG,COVID-19 IgG,Reagente,NaN,Inconclusivo: Índice >= 0.8 e < 1.1;Não reage...
98527,3978f0fe0e14963ca3c95232bdf33532,018c7e14ec0445c170461b744bd7e026,2020-07-08,HOSP,COVID-19 - PESQUISA DE ANTICORPOS IgG,COVID-19 IgG,Reagente,NaN,Inconclusivo: Índice >= 0.8 e < 1.1;Não reage...
1555885,84f5991c67547fa80fe12c0ebaf314c2,0200a7fee74490fcac31fba3c1c3b282,2020-07-01,HOSP,COVID-19 - PESQUISA DE ANTICORPOS IgG,COVID-19 IgG,Não reagente,NaN,Inconclusivo: Índice >= 0.8 e < 1.1;Não reage...
1974654,0b4369f22ba7c2bba489605a28090e42,02a99cab514cf42129c3bbc01a53b3a3,2020-06-18,HOSP,COVID-19 - PESQUISA DE ANTICORPOS IgG,COVID-19 IgG,Reagente,NaN,Inconclusivo: Índice >= 0.8 e < 1.1;Não reage...
1991826,17bd13f07cb8d3d3ebdaa7f885b2e879,033a501926526b396c8672c83a8834f2,2020-06-05,HOSP,COVID-19 - PESQUISA DE ANTICORPOS IgG,COVID-19 IgG,Reagente,NaN,Inconclusivo: Índice >= 0.8 e < 1.1;Não reage...


In [ ]:
attendimento_ids_with_covid = covid_performed_df['ID_aTENDIMENTO'].unique()
all_exams_for_covid_attendances = exames_df_cleaned[exames_df_cleaned['ID_aTENDIMENTO'].isin(attendimento_ids_with_covid)]

print(f"Número de linhas (exames) para os atendimentos com o exame de COVID-19: {len(all_exams_for_covid_attendances)}")
print("\nPrimeiras linhas do dataframe com todos os exames para atendimentos com COVID-19:")
display(all_exams_for_covid_attendances.head())

Número de linhas (exames) para os atendimentos com o exame de COVID-19: 26416

Primeiras linhas do dataframe com todos os exames para atendimentos com COVID-19:


,ID_PACIENTE,ID_aTENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
1173293,d468b9d76f04d517bc4c4140744685ba,016586ae084a86326b913fe74146b102,2020-06-01,HOSP,ALANINA AMINOTRANSFERASE,ALANINA AMINOTRANSFERASE,199,U/L,Homens : inferior a 41 U/L
1173292,d468b9d76f04d517bc4c4140744685ba,016586ae084a86326b913fe74146b102,2020-06-01,HOSP,ASPARTATO AMINO TRANSFERASE,ASPARTATO AMINO TRANSFERASE,74,U/L,Homens : inferior a 37 U/L
1173291,d468b9d76f04d517bc4c4140744685ba,016586ae084a86326b913fe74146b102,2020-06-01,HOSP,BILIRRUBINA DIRETA,BILIRRUBINA DIRETA,0.45,mg/dL,inferior a 0.30 mg/dL
1173290,d468b9d76f04d517bc4c4140744685ba,016586ae084a86326b913fe74146b102,2020-06-01,HOSP,BILIRRUBINA INDIRETA,BILIRRUBINA INDIRETA,0.17,mg/dL,0.10 a 0.60 mg/dL
1173289,d468b9d76f04d517bc4c4140744685ba,016586ae084a86326b913fe74146b102,2020-06-01,HOSP,BILIRRUBINA TOTAL,BILIRRUBINA TOTAL,0.62,mg/dL,Adultos e Crianças: 0.20 a 1.00 mg/dL


In [ ]:
# Pivoteamento
covid_exams_pivot = all_exams_for_covid_attendances.pivot_table(
    index='ID_aTENDIMENTO',
    columns='DE_EXAME',
    values='DE_RESULTADO',
    aggfunc='first'
)

print("Primeiras linhas da tabela pivotada:")
display(covid_exams_pivot.head())

Primeiras linhas da tabela pivotada:


DE_EXAME,25-HIDROXIVITAMINA D,ADENOSINA DEAMINASE,ADENOVIRUS,AGNOR,ALANINA AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA 1 ANTITRIPSINA,ALFA 1 GLICOPROTEÍNA ÁCIDA,ALFAFETOPROTEÍNA,...,VARICELA IgM,VDRL,VELOCIDADE DE HEMOSSEDIMENTAÇÃO,VITAMINA B 12,VLDL - COLESTEROL,VÍRUS SINCICIAL RESPIRATÓRIO,Vírus sincicial respiratório A (HRSVA) e B (HRSVB),ÁCIDO FÓLICO,ÁCIDO VALPRÓICO,ÁCIDO ÚRICO
ID_aTENDIMENTO,,,,,,,,,,,,,,,,,,,,,
016586ae084a86326b913fe74146b102,NaN,NaN,NaN,NaN,199,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
018c7e14ec0445c170461b744bd7e026,NaN,NaN,NaN,NaN,78,2.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,46,NaN,NaN,NaN,NaN,NaN
0200a7fee74490fcac31fba3c1c3b282,NaN,NaN,NaN,NaN,17,2.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02a99cab514cf42129c3bbc01a53b3a3,NaN,NaN,NaN,NaN,106,2.1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,44,NaN,NaN,NaN,NaN,NaN
033a501926526b396c8672c83a8834f2,NaN,NaN,NaN,NaN,37,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Nan -> 0 ou 0.5
covid_exams_pivot_filled_conditional = covid_exams_pivot.copy()

for col in covid_exams_pivot_filled_conditional.columns:
    numeric_col = pd.to_numeric(covid_exams_pivot_filled_conditional[col], errors='coerce')
    numeric_proportion = numeric_col.notna().sum() / covid_exams_pivot_filled_conditional[col].notna().sum() if covid_exams_pivot_filled_conditional[col].notna().sum() > 0 else 0

    if numeric_proportion > 0.8 or (covid_exams_pivot_filled_conditional[col].notna().sum() > 0 and numeric_proportion == 1.0):

        covid_exams_pivot_filled_conditional[col] = covid_exams_pivot_filled_conditional[col].fillna(0)
    else:

        unique_non_null_values = covid_exams_pivot_filled_conditional[col].dropna().unique()
        if len(unique_non_null_values) == 2:

            covid_exams_pivot_filled_conditional[col] = covid_exams_pivot_filled_conditional[col].fillna(0.5)
        else:

            covid_exams_pivot_filled_conditional[col] = covid_exams_pivot_filled_conditional[col].fillna('Não realizado')
display(covid_exams_pivot_filled_conditional.head())


DE_EXAME,25-HIDROXIVITAMINA D,ADENOSINA DEAMINASE,ADENOVIRUS,AGNOR,ALANINA AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA 1 ANTITRIPSINA,ALFA 1 GLICOPROTEÍNA ÁCIDA,ALFAFETOPROTEÍNA,...,VARICELA IgM,VDRL,VELOCIDADE DE HEMOSSEDIMENTAÇÃO,VITAMINA B 12,VLDL - COLESTEROL,VÍRUS SINCICIAL RESPIRATÓRIO,Vírus sincicial respiratório A (HRSVA) e B (HRSVB),ÁCIDO FÓLICO,ÁCIDO VALPRÓICO,ÁCIDO ÚRICO
ID_aTENDIMENTO,,,,,,,,,,,,,,,,,,,,,
016586ae084a86326b913fe74146b102,0,Não realizado,Não realizado,Não realizado,199,0,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,0,Não realizado,Não realizado,0,Não realizado,0
018c7e14ec0445c170461b744bd7e026,0,Não realizado,Não realizado,Não realizado,78,2.9,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,46,Não realizado,Não realizado,0,Não realizado,0
0200a7fee74490fcac31fba3c1c3b282,0,Não realizado,Não realizado,Não realizado,17,2.9,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,0,Não realizado,Não realizado,0,Não realizado,0
02a99cab514cf42129c3bbc01a53b3a3,0,Não realizado,Não realizado,Não realizado,106,2.1,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,44,Não realizado,Não realizado,0,Não realizado,0
033a501926526b396c8672c83a8834f2,0,Não realizado,Não realizado,Não realizado,37,0,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,0,Não realizado,Não realizado,0,Não realizado,0


In [ ]:
#Limpa a tabela
import re

def clean_column_names(df):
    """Cleans column names by removing special characters and spaces."""
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'\s+', '_', col)
        new_col = re.sub(r'[^\w_]+', '', new_col)
        new_col = new_col.strip('_')
        new_cols.append(new_col)
    df.columns = new_cols
    return df

covid_exams_pivot_filled_conditional = clean_column_names(covid_exams_pivot_filled_conditional)

print("Nomes das colunas após a limpeza:")
print(covid_exams_pivot_filled_conditional.columns.tolist())

Nomes das colunas após a limpeza:
['25HIDROXIVITAMINA_D', 'ADENOSINA_DEAMINASE', 'ADENOVIRUS', 'AGNOR', 'ALANINA_AMINOTRANSFERASE', 'ALBUMINA', 'ALDOSTERONA', 'ALFA_1_ANTITRIPSINA', 'ALFA_1_GLICOPROTEÍNA_ÁCIDA', 'ALFAFETOPROTEÍNA', 'AMICACINA', 'AMILASE', 'AMÔNIA', 'ANTI__TRANSGLUTAMINASE_TECIDUAL', 'ANTIESTREPTOLISINA_O_ASLO', 'ANTIBIOGRAMA', 'ANTICORPO_ANTIENDOMISIO', 'ANTICORPO_ANTIMICROSSOMA_DE_FIGADO_E_RIM_ANTILKM1', 'ANTICORPO_ANTIMITOCÔNDRIA', 'ANTICORPO_ANTIMÚSCULO_LISO', 'ANTICORPO_ANTIPEROXIDASE_ANTITPO', 'ANTICORPO_ANTIRECEPTOR_DE_TSH_TRAb', 'ANTICORPO_ANTITIREOGLOBULINA', 'ANTICORPOS_ANTICARDIOLIPINA_IgG', 'ANTICORPOS_ANTICARDIOLIPINA_IgM', 'ANTICORPOS_ANTICITOPLASMA_DE_NEUTRÓFILOS_ANCA', 'ANTICORPOS_ANTIDNA_NATIVO_ANTI_DNA_DE_FITA_DUPLA', 'ANTICORPOS_ANTIRNPSM', 'ANTICORPOS_ANTISCL_70_ANTITOPOISOMERASE_I', 'ANTICORPOS_ANTISM', 'ANTICORPOS_ANTISSARO', 'ANTICORPOS_ANTISSBLA', 'ANTIGENEMIA_PARA_CRYPTOCOCCUS_NEOFORMANS', 'ANTITROMBINA_III', 'ANTÍGENO_CARCINOEMBRIONÁRIO', 'ANTÍ

In [ ]:
display(covid_exams_pivot_filled_conditional.head())

,25HIDROXIVITAMINA_D,ADENOSINA_DEAMINASE,ADENOVIRUS,AGNOR,ALANINA_AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA_1_ANTITRIPSINA,ALFA_1_GLICOPROTEÍNA_ÁCIDA,ALFAFETOPROTEÍNA,...,VARICELA_IgM,VDRL,VELOCIDADE_DE_HEMOSSEDIMENTAÇÃO,VITAMINA_B_12,VLDL__COLESTEROL,VÍRUS_SINCICIAL_RESPIRATÓRIO,Vírus_sincicial_respiratório_A_HRSVA_e_B_HRSVB,ÁCIDO_FÓLICO,ÁCIDO_VALPRÓICO,ÁCIDO_ÚRICO
ID_aTENDIMENTO,,,,,,,,,,,,,,,,,,,,,
016586ae084a86326b913fe74146b102,0,Não realizado,Não realizado,Não realizado,199,0,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,0,Não realizado,Não realizado,0,Não realizado,0
018c7e14ec0445c170461b744bd7e026,0,Não realizado,Não realizado,Não realizado,78,2.9,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,46,Não realizado,Não realizado,0,Não realizado,0
0200a7fee74490fcac31fba3c1c3b282,0,Não realizado,Não realizado,Não realizado,17,2.9,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,0,Não realizado,Não realizado,0,Não realizado,0
02a99cab514cf42129c3bbc01a53b3a3,0,Não realizado,Não realizado,Não realizado,106,2.1,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,44,Não realizado,Não realizado,0,Não realizado,0
033a501926526b396c8672c83a8834f2,0,Não realizado,Não realizado,Não realizado,37,0,Não realizado,0,0,0,...,Não realizado,Não realizado,0,0,0,Não realizado,Não realizado,0,Não realizado,0


In [ ]:
covid_exams_pivot_filled_conditional.replace('Não realizado', 0, inplace=True)
display(covid_exams_pivot_filled_conditional.head())

,25HIDROXIVITAMINA_D,ADENOSINA_DEAMINASE,ADENOVIRUS,AGNOR,ALANINA_AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA_1_ANTITRIPSINA,ALFA_1_GLICOPROTEÍNA_ÁCIDA,ALFAFETOPROTEÍNA,...,VARICELA_IgM,VDRL,VELOCIDADE_DE_HEMOSSEDIMENTAÇÃO,VITAMINA_B_12,VLDL__COLESTEROL,VÍRUS_SINCICIAL_RESPIRATÓRIO,Vírus_sincicial_respiratório_A_HRSVA_e_B_HRSVB,ÁCIDO_FÓLICO,ÁCIDO_VALPRÓICO,ÁCIDO_ÚRICO
ID_aTENDIMENTO,,,,,,,,,,,,,,,,,,,,,
016586ae084a86326b913fe74146b102,0,0,0,0,199,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
018c7e14ec0445c170461b744bd7e026,0,0,0,0,78,2.9,0,0,0,0,...,0,0,0,0,46,0,0,0,0,0
0200a7fee74490fcac31fba3c1c3b282,0,0,0,0,17,2.9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02a99cab514cf42129c3bbc01a53b3a3,0,0,0,0,106,2.1,0,0,0,0,...,0,0,0,0,44,0,0,0,0,0
033a501926526b396c8672c83a8834f2,0,0,0,0,37,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Reagente por 1 e não reagente por 0
covid_column_name = None
for col in covid_exams_pivot_filled_conditional.columns:
    if 'COVID19' in col.upper() or 'SARSCOV2' in col.upper() or 'TESTE_RAPIDO_PARA_COVID19' in col.upper():
        covid_column_name = col
        break

if covid_column_name:
    covid_exams_pivot_filled_conditional[covid_column_name] = covid_exams_pivot_filled_conditional[covid_column_name].replace({
        'Reagente': 1,
        'Não reagente': 0
    })
    display(covid_exams_pivot_filled_conditional[covid_column_name].head())
else:
    print("Não foi possível encontrar a coluna do exame de COVID-19 na tabela para realizar a substituição.")

,COVID19__PESQUISA_DE_ANTICORPOS_IgG
ID_aTENDIMENTO,
016586ae084a86326b913fe74146b102,1
018c7e14ec0445c170461b744bd7e026,1
0200a7fee74490fcac31fba3c1c3b282,0
02a99cab514cf42129c3bbc01a53b3a3,1
033a501926526b396c8672c83a8834f2,1


In [ ]:
display(covid_exams_pivot_filled_conditional.head())

,25HIDROXIVITAMINA_D,ADENOSINA_DEAMINASE,ADENOVIRUS,AGNOR,ALANINA_AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA_1_ANTITRIPSINA,ALFA_1_GLICOPROTEÍNA_ÁCIDA,ALFAFETOPROTEÍNA,...,VARICELA_IgM,VDRL,VELOCIDADE_DE_HEMOSSEDIMENTAÇÃO,VITAMINA_B_12,VLDL__COLESTEROL,VÍRUS_SINCICIAL_RESPIRATÓRIO,Vírus_sincicial_respiratório_A_HRSVA_e_B_HRSVB,ÁCIDO_FÓLICO,ÁCIDO_VALPRÓICO,ÁCIDO_ÚRICO
ID_aTENDIMENTO,,,,,,,,,,,,,,,,,,,,,
016586ae084a86326b913fe74146b102,0,0,0,0,199,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
018c7e14ec0445c170461b744bd7e026,0,0,0,0,78,2.9,0,0,0,0,...,0,0,0,0,46,0,0,0,0,0
0200a7fee74490fcac31fba3c1c3b282,0,0,0,0,17,2.9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02a99cab514cf42129c3bbc01a53b3a3,0,0,0,0,106,2.1,0,0,0,0,...,0,0,0,0,44,0,0,0,0,0
033a501926526b396c8672c83a8834f2,0,0,0,0,37,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Criar o DataFrame attendance_patient_map com ID_aTENDIMENTO e ID_PACIENTE
attendance_patient_map = exames_df_cleaned[['ID_aTENDIMENTO', 'ID_PACIENTE']].drop_duplicates()
display(attendance_patient_map.head())

,ID_aTENDIMENTO,ID_PACIENTE
1281318,005dc13e19b2523e0db4bf15156d6412,f88a18f43c87d534877bda69ff89b068
1469225,00759fffd8c3fcd6a790e67a8bbfcb59,59ce0f1d16cdc3f1889442a96979ad3c
383064,00a495aa8cdc20aa8272e2ecab6a1c28,b9708efa3f3a88cb481970173ed33877
2383530,00b4a366c7da2ce894a4fa7a2b1c45bb,ce5d1c92d88870d43288548755015d26
1247275,00b8650bb74e3c650560486ad30a5898,ece33041542846f6aff1017fea989836


In [ ]:
attendance_patient_info = pd.merge(attendance_patient_map, pacientes_df[['ID_PACIENTE', 'AA_NASCIMENTO', 'IC_SEXO']], on='ID_PACIENTE', how='left')
display(attendance_patient_info.head())

,ID_aTENDIMENTO,ID_PACIENTE,AA_NASCIMENTO,IC_SEXO
0,005dc13e19b2523e0db4bf15156d6412,f88a18f43c87d534877bda69ff89b068,1969,M
1,00759fffd8c3fcd6a790e67a8bbfcb59,59ce0f1d16cdc3f1889442a96979ad3c,1951,M
2,00a495aa8cdc20aa8272e2ecab6a1c28,b9708efa3f3a88cb481970173ed33877,1941,F
3,00b4a366c7da2ce894a4fa7a2b1c45bb,ce5d1c92d88870d43288548755015d26,1956,M
4,00b8650bb74e3c650560486ad30a5898,ece33041542846f6aff1017fea989836,1942,M


In [ ]:
covid_exams_final = pd.merge(covid_exams_pivot_filled_conditional, attendance_patient_info, on='ID_aTENDIMENTO', how='left')
display(covid_exams_final.head())

,ID_aTENDIMENTO,25HIDROXIVITAMINA_D,ADENOSINA_DEAMINASE,ADENOVIRUS,AGNOR,ALANINA_AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA_1_ANTITRIPSINA,ALFA_1_GLICOPROTEÍNA_ÁCIDA,...,VITAMINA_B_12,VLDL__COLESTEROL,VÍRUS_SINCICIAL_RESPIRATÓRIO,Vírus_sincicial_respiratório_A_HRSVA_e_B_HRSVB,ÁCIDO_FÓLICO,ÁCIDO_VALPRÓICO,ÁCIDO_ÚRICO,ID_PACIENTE,AA_NASCIMENTO,IC_SEXO
0,016586ae084a86326b913fe74146b102,0,0,0,0,199,0,0,0,0,...,0,0,0,0,0,0,0,d468b9d76f04d517bc4c4140744685ba,1979,M
1,018c7e14ec0445c170461b744bd7e026,0,0,0,0,78,2.9,0,0,0,...,0,46,0,0,0,0,0,3978f0fe0e14963ca3c95232bdf33532,1974,M
2,0200a7fee74490fcac31fba3c1c3b282,0,0,0,0,17,2.9,0,0,0,...,0,0,0,0,0,0,0,84f5991c67547fa80fe12c0ebaf314c2,1985,F
3,02a99cab514cf42129c3bbc01a53b3a3,0,0,0,0,106,2.1,0,0,0,...,0,44,0,0,0,0,0,0b4369f22ba7c2bba489605a28090e42,1962,F
4,033a501926526b396c8672c83a8834f2,0,0,0,0,37,0,0,0,0,...,0,0,0,0,0,0,0,17bd13f07cb8d3d3ebdaa7f885b2e879,1958,M


In [ ]:
# Tira o ID do paciente
covid_exams_final = covid_exams_final.drop('ID_PACIENTE', axis=1)

print("Primeiras linhas da tabela final após remover a coluna ID_PACIENTE:")
display(covid_exams_final.head())

Primeiras linhas da tabela final após remover a coluna ID_PACIENTE:


,ID_aTENDIMENTO,25HIDROXIVITAMINA_D,ADENOSINA_DEAMINASE,ADENOVIRUS,AGNOR,ALANINA_AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA_1_ANTITRIPSINA,ALFA_1_GLICOPROTEÍNA_ÁCIDA,...,VELOCIDADE_DE_HEMOSSEDIMENTAÇÃO,VITAMINA_B_12,VLDL__COLESTEROL,VÍRUS_SINCICIAL_RESPIRATÓRIO,Vírus_sincicial_respiratório_A_HRSVA_e_B_HRSVB,ÁCIDO_FÓLICO,ÁCIDO_VALPRÓICO,ÁCIDO_ÚRICO,AA_NASCIMENTO,IC_SEXO
0,016586ae084a86326b913fe74146b102,0,0,0,0,199,0,0,0,0,...,0,0,0,0,0,0,0,0,1979,M
1,018c7e14ec0445c170461b744bd7e026,0,0,0,0,78,2.9,0,0,0,...,0,0,46,0,0,0,0,0,1974,M
2,0200a7fee74490fcac31fba3c1c3b282,0,0,0,0,17,2.9,0,0,0,...,0,0,0,0,0,0,0,0,1985,F
3,02a99cab514cf42129c3bbc01a53b3a3,0,0,0,0,106,2.1,0,0,0,...,0,0,44,0,0,0,0,0,1962,F
4,033a501926526b396c8672c83a8834f2,0,0,0,0,37,0,0,0,0,...,0,0,0,0,0,0,0,0,1958,M


In [ ]:
# Identifica linhas não numéricas
non_numeric_rows = pd.DataFrame()

for col in covid_exams_final.columns:
    numeric_col = pd.to_numeric(covid_exams_final[col], errors='coerce')
    non_numeric_mask = numeric_col.isna() & covid_exams_final[col].notna()

    if non_numeric_mask.any():
        print(f"Linhas com valores não numéricos na coluna '{col}':")
        display(covid_exams_final[non_numeric_mask])
        non_numeric_rows = pd.concat([non_numeric_rows, covid_exams_final[non_numeric_mask]])


In [ ]:
# Esta célula substitui os valores 'Negativa' e 'Indetectavel' por 0
exclude_cols_for_replace = ['ID_aTENDIMENTO', 'ID_PACIENTE', 'AA_NASCIMENTO', 'IC_SEXO']
data_cols_for_replace = [col for col in covid_exams_final.columns if col not in exclude_cols_for_replace]
count_negativa_before = (covid_exams_final[data_cols_for_replace] == 'Negativa').sum().sum()
count_indetectavel_before = (covid_exams_final[data_cols_for_replace] == 'Indetectavel').sum().sum()
modified_count = count_negativa_before + count_indetectavel_before

covid_exams_final[data_cols_for_replace] = covid_exams_final[data_cols_for_replace].replace(['Negativa', 'Indetectavel'], 0)

display(covid_exams_final.head())


/tmp/ipython-input-50-271020564.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  covid_exams_final[data_cols_for_replace] = covid_exams_final[data_cols_for_replace].replace(['Negativa', 'Indetectavel'], 0)


,ID_aTENDIMENTO,25HIDROXIVITAMINA_D,ADENOSINA_DEAMINASE,ADENOVIRUS,AGNOR,ALANINA_AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA_1_ANTITRIPSINA,ALFA_1_GLICOPROTEÍNA_ÁCIDA,...,VELOCIDADE_DE_HEMOSSEDIMENTAÇÃO,VITAMINA_B_12,VLDL__COLESTEROL,VÍRUS_SINCICIAL_RESPIRATÓRIO,Vírus_sincicial_respiratório_A_HRSVA_e_B_HRSVB,ÁCIDO_FÓLICO,ÁCIDO_VALPRÓICO,ÁCIDO_ÚRICO,AA_NASCIMENTO,IC_SEXO
0,016586ae084a86326b913fe74146b102,0,0,0,0,199,0,0,0,0,...,0,0,0,0,0,0,0,0,1979,M
1,018c7e14ec0445c170461b744bd7e026,0,0,0,0,78,2.9,0,0,0,...,0,0,46,0,0,0,0,0,1974,M
2,0200a7fee74490fcac31fba3c1c3b282,0,0,0,0,17,2.9,0,0,0,...,0,0,0,0,0,0,0,0,1985,F
3,02a99cab514cf42129c3bbc01a53b3a3,0,0,0,0,106,2.1,0,0,0,...,0,0,44,0,0,0,0,0,1962,F
4,033a501926526b396c8672c83a8834f2,0,0,0,0,37,0,0,0,0,...,0,0,0,0,0,0,0,0,1958,M


In [ ]:
# Esta célula tenta converter as colunas de resultados de exames para numérico,

exclude_cols_for_numeric = ['ID_aTENDIMENTO', 'ID_PACIENTE', 'AA_NASCIMENTO', 'IC_SEXO']
cols_to_convert = [col for col in covid_exams_final.columns if col not in exclude_cols_for_numeric]

for col in cols_to_convert:
    covid_exams_final[col] = pd.to_numeric(covid_exams_final[col], errors='coerce')

In [ ]:
# Esta célula substitui os valores 'M' por 1 e 'F' por 0 na coluna 'IC_SEXO'.
covid_exams_final['IC_SEXO'] = covid_exams_final['IC_SEXO'].replace({'M': 1, 'F': 0})
display(covid_exams_final['IC_SEXO'].head())

,IC_SEXO
0,1
1,1
2,0
3,0
4,1


In [ ]:
covid_exams_final.head()

,ID_aTENDIMENTO,25HIDROXIVITAMINA_D,ADENOSINA_DEAMINASE,ADENOVIRUS,AGNOR,ALANINA_AMINOTRANSFERASE,ALBUMINA,ALDOSTERONA,ALFA_1_ANTITRIPSINA,ALFA_1_GLICOPROTEÍNA_ÁCIDA,...,VELOCIDADE_DE_HEMOSSEDIMENTAÇÃO,VITAMINA_B_12,VLDL__COLESTEROL,VÍRUS_SINCICIAL_RESPIRATÓRIO,Vírus_sincicial_respiratório_A_HRSVA_e_B_HRSVB,ÁCIDO_FÓLICO,ÁCIDO_VALPRÓICO,ÁCIDO_ÚRICO,AA_NASCIMENTO,IC_SEXO
0,016586ae084a86326b913fe74146b102,0.0,0.0,0,0.0,199,0.0,0.0,0,0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1979,1
1,018c7e14ec0445c170461b744bd7e026,0.0,0.0,0,0.0,78,2.9,0.0,0,0,...,0,0.0,46,0.0,0.0,0.0,0.0,0.0,1974,1
2,0200a7fee74490fcac31fba3c1c3b282,0.0,0.0,0,0.0,17,2.9,0.0,0,0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1985,0
3,02a99cab514cf42129c3bbc01a53b3a3,0.0,0.0,0,0.0,106,2.1,0.0,0,0,...,0,0.0,44,0.0,0.0,0.0,0.0,0.0,1962,0
4,033a501926526b396c8672c83a8834f2,0.0,0.0,0,0.0,37,0.0,0.0,0,0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1958,1


In [ ]:
print("Contagem de valores NaN por coluna antes da substituição:")
display(covid_exams_final.isnull().sum())

covid_exams_final.fillna(0, inplace=True)

print("\nContagem de valores NaN por coluna após a substituição:")
display(covid_exams_final.isnull().sum())

Contagem de valores NaN por coluna antes da substituição:


,0
ID_aTENDIMENTO,0
25HIDROXIVITAMINA_D,0
ADENOSINA_DEAMINASE,0
ADENOVIRUS,0
AGNOR,0
...,...
ÁCIDO_FÓLICO,0
ÁCIDO_VALPRÓICO,0
ÁCIDO_ÚRICO,0
AA_NASCIMENTO,0



Contagem de valores NaN por coluna após a substituição:


,0
ID_aTENDIMENTO,0
25HIDROXIVITAMINA_D,0
ADENOSINA_DEAMINASE,0
ADENOVIRUS,0
AGNOR,0
...,...
ÁCIDO_FÓLICO,0
ÁCIDO_VALPRÓICO,0
ÁCIDO_ÚRICO,0
AA_NASCIMENTO,0


In [ ]:
# Esta célula exibe o número de linhas (exemplos) no dataframe covid_exams_final.
num_examples = len(covid_exams_final)

print(f"A tabela final possui {num_examples} exemplos (linhas).")

A tabela final possui 554 exemplos (linhas).


In [ ]:
# Esta célula substitui os valores 'AAAA' por '1980' na coluna 'AA_NASCIMENTO'.
covid_exams_final['AA_NASCIMENTO'] = covid_exams_final['AA_NASCIMENTO'].replace('AAAA', '1980')
display(covid_exams_final['AA_NASCIMENTO'].head())

,AA_NASCIMENTO
0,1979
1,1974
2,1985
3,1962
4,1958


In [ ]:
# Esta célula salva o dataframe final 'covid_exams_final' em um arquivo CSV
covid_exams_final.to_csv('covid_exams_final.csv', index=False)

In [ ]:
# Esta célula exibe o número de colunas no dataframe covid_exams_final.
num_columns = covid_exams_final.shape[1]

print(f"A tabela final possui {num_columns} colunas.")

A tabela final possui 361 colunas.
